## Solution 1: Rank importtance based on impurity decrease 

The working mechanism of random forest is to reduce the impurity. Impurity is computed based two main measurements Gini impurity of information gain. So the feature importance is ranked by the decrease of impurity decrease.

Follow is an example.

In [4]:
from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
import numpy as np

#Load boston housing dataset as an example
boston = load_boston()
X = boston["data"]
Y = boston["target"]
names = boston["feature_names"]
rf = RandomForestRegressor()

rf.fit(X, Y)
print ("Features sorted by their score:")
print (sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_), names), 
             reverse=True))

Features sorted by their score:
[(0.4769, 'RM'), (0.3309, 'LSTAT'), (0.0672, 'DIS'), (0.0357, 'CRIM'), (0.0202, 'NOX'), (0.0193, 'PTRATIO'), (0.0143, 'AGE'), (0.0125, 'TAX'), (0.0119, 'B'), (0.0079, 'INDUS'), (0.0017, 'RAD'), (0.0008, 'CHAS'), (0.0005, 'ZN')]


## Solution 2: Directly measure the impact of each feature on accuracy of the model

We permute the values of each feature and measure how much the permutation decreases the accuracy of the model. Clearly, for unimportant variables, the permutation should have little to no effect on model accuracy, while permuting important variables should significantly decrease it.

Follow is an example.

In [6]:
from sklearn.cross_validation import ShuffleSplit
from sklearn.metrics import r2_score
from collections import defaultdict
 
X = boston["data"]
Y = boston["target"]
 
rf = RandomForestRegressor()
scores = defaultdict(list)
 
#crossvalidate the scores on a number of different random splits of the data
for train_idx, test_idx in ShuffleSplit(len(X), 100, .3):
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]
    r = rf.fit(X_train, Y_train)
    acc = r2_score(Y_test, rf.predict(X_test))
    for i in range(X.shape[1]):
        X_t = X_test.copy()
    # permuate all the features
        np.random.shuffle(X_t[:, i])
        shuff_acc = r2_score(Y_test, rf.predict(X_t))
        scores[names[i]].append((acc-shuff_acc)/acc)
print ("Features sorted by their score:")
print (sorted([(round(np.mean(score), 4), feat) for
              feat, score in scores.items()], reverse=True))


c:\users\xuchen\appdata\local\programs\python\python36\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Features sorted by their score:
[(0.7716, 'LSTAT'), (0.5453, 'RM'), (0.0898, 'DIS'), (0.0386, 'NOX'), (0.0376, 'CRIM'), (0.0226, 'PTRATIO'), (0.0156, 'TAX'), (0.0115, 'AGE'), (0.006, 'B'), (0.0049, 'INDUS'), (0.0033, 'RAD'), (0.0002, 'CHAS'), (0.0001, 'ZN')]


LSTAT and RM are the most important features. The conclusion is very close to the above ones.